In [ ]:
from to_json import to_json
from gdsfactory.pdk import get_layer_stack
import gdsfactory as gf

In [ ]:
from sky130 import LAYER_STACK

c = gf.read.import_gds("example_sky130.gds")
layer_stack = LAYER_STACK
name = "sky130"
# j = to_json(c, layer_stack=layer_stack, return_json=False)

In [ ]:
from typing import cast
from sky130 import LAYER_STACK

from kfactory import LayerEnum

from gdsfactory.component import Component
from gdsfactory.technology import DerivedLayer, LayerStack, LayerViews, LogicalLayer
from gdsfactory.typings import LayerSpecs
from gdsfactory.pdk import (
    get_active_pdk,
    get_layer,
    get_layer_stack,
    get_layer_views,
)

import gdsfactory as gf


def to_poly(
    component: Component,
    layer_views: LayerViews | None = None,
    layer_stack: LayerStack | None = None,
    exclude_layers: LayerSpecs | None = None,
    return_json: bool = True,
):
    """Return optimzed json.

    Args:
        component: to extrude in 3D.
        layer_views: layer colors from Klayout Layer Properties file.
            Defaults to active PDK.layer_views.
        layer_stack: contains thickness and zmin for each layer.
            Defaults to active PDK.layer_stack.
        exclude_layers: list of layer index to exclude.

    """

    layer_views = layer_views or get_layer_views()
    layer_stack = layer_stack or get_layer_stack()

    exclude_layers = exclude_layers or ()
    exclude_layers = [get_layer(layer) for layer in exclude_layers]

    component_with_booleans = layer_stack.get_component_with_derived_layers(component)
    polygons_per_layer = component_with_booleans.get_polygons_points(merge=True)
    has_polygons = False

    top_name = "GDS"

    polygons = {}
    
    for level in layer_stack.layers.values():
        layer = level.layer

        if isinstance(layer, LogicalLayer):
            assert isinstance(layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(layer.layer))
        elif isinstance(layer, DerivedLayer):
            assert level.derived_layer is not None
            assert isinstance(level.derived_layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(level.derived_layer.layer))
        else:
            raise ValueError(f"Layer {layer!r} is not a DerivedLayer or LogicalLayer")

        layer_index = int(get_layer(layer_tuple))

        if layer_index in exclude_layers:
            continue

        if layer_index not in polygons_per_layer:
            continue

        zmin = level.zmin
        height = level.thickness
        layer_view = layer_views.get_from_tuple(layer_tuple)

        assert layer_view.fill_color is not None
        if zmin is not None and layer_view.visible:
            has_polygons = True
            polygons[layer_view.name] = polygons_per_layer[layer_index]

    if not has_polygons:
        raise ValueError(
            f"{component.name!r} does not have polygons defined in the "
            f"layer_stack or layer_views for the active Pdk {get_active_pdk().name!r}"
        )

    return polygons


In [ ]:
polygons = to_poly(c, layer_stack=LAYER_STACK)

In [ ]:
polygons.keys()

In [ ]:
polygon = polygons['polydrawing_m']

In [ ]:
import plotly.graph_objects as go
import numpy as np

def plot_polygons(polygons, width=1300, height=1300):
    """
    Plot all original polygons in one interactive Plotly figure.
    """
    fig = go.Figure()
    # colors = [f"rgba({100+30*i%155},{100+60*i%155},{200-30*i%155},1)" for i in range(len(polygons))]
    colors = [f"rgba(0.3, 0.3, 0.3, 1)" for i in range(len(polygons))]
    fill_colors = [f"rgba(0.3, 0.3, 0.3, 0.2)" for i in range(len(polygons))]
    for i, poly in enumerate(polygons):
        poly = np.array(poly)
        poly_closed = np.vstack([poly, poly[0]])
        fig.add_trace(go.Scatter(
            x=poly_closed[:, 0], y=poly_closed[:, 1],
            mode='lines',
            name=f"Original {i}",
            line=dict(width=2, color=colors[i]),
            fill='toself',
            fillcolor=fill_colors[i]
        ))
    fig.update_layout(
        title="All Original Polygons",
        width=width, height=height,
        xaxis=dict(scaleanchor="y", scaleratio=1, showgrid=False, zeroline=False),
        yaxis=dict(scaleanchor="x", scaleratio=1, showgrid=False, zeroline=False),
        showlegend=False,
        plot_bgcolor="#FFF",
    )

    fig.show()

plot_polygons(polygon)


In [ ]:
import numpy as np
from numba import njit
from collections import defaultdict

# Precomputed inverse transformation indices [0-7]
INVERSE_MAP = [0, 3, 2, 1, 4, 7, 6, 5]

#@njit
def translate_to_centroid(points):
    centroid = np.mean(points, axis=0)
    return points - centroid

#@njit
def rotate90(points):
    rotated = np.empty_like(points)
    rotated[:, 0] = points[:, 1]
    rotated[:, 1] = -points[:, 0]
    return rotated

# @njit
def rotate180(points):
    return -points

# @njit
def rotate270(points):
    rotated = np.empty_like(points)
    rotated[:, 0] = -points[:, 1]
    rotated[:, 1] = points[:, 0]
    return rotated

# @njit
def reflect(points):
    reflected = np.empty_like(points)
    reflected[:, 0] = points[:, 0]
    reflected[:, 1] = -points[:, 1]
    return reflected
    
#@njit
def apply_transformation(points, trans_idx):
    if trans_idx == 0: return points
    elif trans_idx == 1: return rotate90(points)
    elif trans_idx == 2: return rotate180(points)
    elif trans_idx == 3: return rotate270(points)
    elif trans_idx == 4: return reflect(points)
    elif trans_idx == 5: return reflect(rotate90(points))
    elif trans_idx == 6: return reflect(rotate180(points))
    elif trans_idx == 7: return reflect(rotate270(points))

# @njit
def normalize_and_sort(points):
    """Centroid normalization + vertex sorting"""
    centroid = np.mean(points, axis=0)
    normalized = points - centroid
    order = np.lexsort((normalized[:, 1], normalized[:, 0]))
    return normalized[order]

def group_congruent_polygons(polygons):
    length_groups = defaultdict(list)
    
    for idx, poly in enumerate(polygons):
        poly_array = np.array(poly, dtype=np.float64)
        
        # Generate all possible canonical forms
        min_key = None
        for trans_idx in range(8):
            transformed = apply_transformation(poly_array, trans_idx)
            processed = normalize_and_sort(transformed)
            key = tuple(np.round(processed.flatten(), 6))
            
            if (min_key is None) or (key < min_key):
                min_key = key
                best_trans = trans_idx
                best_processed = processed
        
        length_groups[(len(poly_array), min_key)].append(
            (idx, best_processed, best_trans)
        )

    # Build groups with reconstruction data
    groups = []
    for (vertex_count, key), members in length_groups.items():
        ref_poly = members[0][1]  # Already normalized and sorted
        group_entry = {
            'reference': ref_poly.tolist(),
            'members': []
        }
        
        for mem_idx, processed_poly, trans_idx in members:
            # Calculate inverse transformation to reach original
            inverse_trans = INVERSE_MAP[trans_idx]
            centroid = np.mean(polygons[mem_idx], axis=0)
            
            group_entry['members'].append({
                'original_index': mem_idx,
                'centroid': centroid.tolist(),
                'transformation': inverse_trans
            })
        
        groups.append(group_entry)
    
    return groups

def reconstruct_from_group(group, member, apply_transformation):
    ref_poly = np.array(group['reference'])
    transformed = apply_transformation(ref_poly, member['transformation'])
    return transformed + np.array(member['centroid'])



In [ ]:
groups = group_congruent_polygons(polygon)

In [ ]:
def plot_polygon_groups(groups, apply_transformation, width=900, height=700):
    """
    Plot each congruence group with its canonical reference and all reconstructed members.
    """
    for group_idx, group in enumerate(groups):
        fig = go.Figure()
        # Reference polygon (canonical form at origin)
        ref_poly = np.array(group['reference'])
        ref_closed = np.vstack([ref_poly, ref_poly[0]])
        fig.add_trace(go.Scatter(
            x=ref_closed[:, 0], y=ref_closed[:, 1],
            mode='lines+markers',
            name='Canonical Reference',
            line=dict(color='black', width=3, dash='dash'),
            marker=dict(color='black', size=6),
            fill='toself',
            fillcolor='rgba(50,50,50,0.1)'
        ))

        # All reconstructed group members
        # Use a fixed color cycle for consistency
        colors = [f"rgba({100+30*i%155},{100+60*i%155},{200-30*i%155},1.0)" for i in range(len(group['members']))]
        fill_colors = [f"rgba({100+30*i%155},{100+60*i%155},{200-30*i%155},0.2)" for i in range(len(group['members']))]
        for i, member in enumerate(group['members']):
            reconstructed = reconstruct_from_group(group, member, apply_transformation)
            reconstructed = np.array(reconstructed)
            reconstructed_closed = np.vstack([reconstructed, reconstructed[0]])
            fig.add_trace(go.Scatter(
                x=reconstructed_closed[:, 0], y=reconstructed_closed[:, 1],
                mode='lines',
                name=f"Original {member['original_index']}",
                line=dict(width=2, color=colors[i % len(colors)]),
                marker=dict(size=6, color=colors[i % len(colors)]),
                fill='toself',
                fillcolor=colors[i % len(colors)]
            ))

        fig.update_layout(
            title=f'Group {group_idx} ({len(group["members"])} members)',
            width=width, height=height,
            xaxis=dict(scaleanchor="y", scaleratio=1, showgrid=False, zeroline=False),
            yaxis=dict(scaleanchor="x", scaleratio=1, showgrid=False, zeroline=False),
            showlegend=False,
            plot_bgcolor="#FFF",
        )
        fig.show()



In [ ]:
plot_polygon_groups(groups, apply_transformation, width=1300, height=1300)

In [ ]:
plot_polygon_groups(groups, apply_transformation)

In [ ]:
c = gf.c.straight_heater_doped_rib(length=100)
layer_stack = get_layer_stack()
j = to_json(c, layer_stack=layer_stack, return_json=False)

In [ ]:
polygons = [part for part in j["parts"] if part["name"] == "VIA1"][0]["shape"]["polygons"]

In [ ]:
import numpy as np

def group_translation_congruent_polygons(polygons):
    # Round coordinates and convert to uniform float32 dtype
    polygons_rounded = [np.round(p.astype(np.float32), 4) for p in polygons]

    # Calculate centroids using list comprehension
    centroids = np.array([np.mean(p, axis=0) for p in polygons_rounded])

    # Normalize using broadcasting-friendly format
    normalized = [p - c for p, c in zip(polygons_rounded, centroids)]

    # Create hashable keys with length encoding
    norm_keys = [tuple([len(p)] + p.ravel().round(3).tolist()) for p in normalized]

    # Group with dictionary comprehension
    groups = {}
    for idx, key in enumerate(norm_keys):
        if key not in groups:
            groups[key] = {
                "polygon": normalized[idx].ravel(),
                "offsets": [],
            }
        groups[key]["offsets"].extend(centroids[idx].tolist())
    for key in groups.keys():
        groups[key]["offsets"] = np.asarray(groups[key]["offsets"], dtype="float32")
    return list(groups.values())

groups = group_translation_congruent_polygons(polygons)

In [ ]:
polygons[0]

In [ ]:
groups[0]["polygon"].reshape(-1,2) + groups[0]["offsets"][:2]

In [ ]:
c.to_3d().show()